In [ ]:
!pip install pandas transformers guidance numpy evaluate rouge-score nltk py-rouge accelerate

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import json
import nltk
import numpy as np
import csv
import warnings
from typing import Any, Dict, Tuple
import re
from transformers import AutoTokenizer, pipeline


warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

# Dataset

In [3]:
from datasets import load_dataset

#load the test data for the generation step for all language pairs
data = load_dataset("Eloquent/HalluciGen-Translation", "test_generation")


Some datasets params were ignored: ['sep']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


Generating test_generation_de_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_en_de split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_fr_en split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test_generation_en_fr split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
df_de_en = data["test_generation_de_en"].to_pandas()
df_de_en.head()

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."


In [5]:
len(df_de_en)

100

In [ ]:
#load model Llama3
model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
def generate_translation(source, target_language):
    # Generate the prompt

    answer_format = {"hyp+": "", "hyp-": ""}

    # prompt for translation generation task
    user_prompt = f'''
    Given the 'src' below, generate a translation hypothesis 'hyp+' that is supported by 'src' and a second translation 'hyp-' that is not supported by 'src'.
    Provide the result in the following format: {answer_format}. Target language: {target_language}

    Src: {source}

    Result:
    '''

    messages = [
      {"role": "system", "content": "You are a text generator. You are specialized in translating texts."},
      {"role": "user", "content": user_prompt},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=250,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    #print("Gen text: ", outputs[0]["generated_text"][len(prompt):])
    generated_paraphrase = outputs[0]["generated_text"][len(prompt):]

    return generated_paraphrase

In [ ]:
import ast
def extract_json_data(result_text):
    # Find the start and end indices of the JSON data
    start_index = result_text.find('{')
    end_index = result_text.find('}') + 1  # Add 1 to include the closing brace

    # Extract the JSON data from the string
    result_text = result_text[start_index:end_index]
    #print("result text: ", result_text)

    # Replace single quotes used as delimiters with double quotes
    result_text = json.loads(result_text.replace("'", "\""))
    return result_text

## Generate dataset

In [ ]:

import os
import numpy as np
import pandas as pd

def save_results(target_language, df, csv_filename):
    skipped_rows = []
    # Split DataFrame into chunks of 10 rows
    chunks = np.array_split(df, len(df) // 10 if len(df) % 10 == 0 else len(df) // 10 + 1)
    for chunk in chunks:
        # Iterate through each row in the chunk
        for index, row in chunk.iterrows():
            source = row['source']

            try:
                # Generate paraphrase
                result = generate_translation(source, target_language)
                #print("source: ", source)
                extracted_hyp = extract_json_data(result)

                hyp_support = extracted_hyp['hyp+']
                #print("hyp+: ", hyp_support)
                hyp_hallu = extracted_hyp['hyp-']
                #print("hyp-: ", hyp_hallu)

                # Update DataFrame with new values
                chunk.at[index, 'hyp+'] = hyp_support
                chunk.at[index, 'hyp-'] = hyp_hallu
            except:
                print(f"Skipping row {index} due to error in data extraction.")
                skipped_rows.append(index)
                continue

        # Append chunk to CSV file
        if not os.path.isfile(csv_filename):
            chunk.to_csv(csv_filename, mode='w', header=True, index=False)
            print("finished chunk")
        else:
            chunk.to_csv(csv_filename, mode='a', header=False, index=False)
            print("finished chunk")

        # Load CSV file
        df = pd.read_csv(csv_filename)
        display(df)
    return skipped_rows

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_de_eng.csv"
skipped_rows = save_results("English", df_de_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 3 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 7 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

finished chunk


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...",Luxury guesthouses are now a common occurrence...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in Britain, but the Sc...","This is common practice in Britain, but the Sc..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...",This fossil is the key to understanding the ti...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...",NaN,NaN
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",A minesweeper of the Avenger-class was en rout...,The minesweeper was on a mission to destroy en...
...,...,...,...,...,...
85,85,de-en,Wenn `` R '' die zugehörige Äquivalenzrelation...,NaN,NaN
86,86,de-en,"Es zeigt 362 verschiedene alte Holzarten, Büsc...",The display shows 362 different old wood speci...,The display shows 362 different old wood speci...
87,87,de-en,"Obrovac ist eine Stadt in Norddalmatien, in Kr...","Obrovac is a city in northern Dalmatia, in Cro...","Obrovac is a city in southern Italy, in the Ro..."
88,88,de-en,Danach wechselte er 1986 zu The Star und verli...,"After that, he joined The Star in 1986 and lef...",He then became a Muslim in 1986 and left The S...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...",Luxury guesthouses are now a common occurrence...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in Britain, but the Sc...","This is common practice in Britain, but the Sc..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...",This fossil is the key to understanding the ti...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...",NaN,NaN
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",A minesweeper of the Avenger-class was en rout...,The minesweeper was on a mission to destroy en...
...,...,...,...,...,...
95,95,de-en,Die Anzahl der gemeldeten Brände betrug Anfang...,The number of reported fires was 73 at the beg...,The number of reported fires was 73 at the beg...
96,96,de-en,Der Ponoru ist eine Nebenfluss des Horezu in R...,The Ponoru is a tributary of the Horezu in Rom...,The Ponoru is a main river of the Horezu in Hu...
97,97,de-en,Leslie Waddington eröffnete 1966 seine eigene ...,Leslie Waddington opened his own gallery in Co...,Leslie Waddington founded his own gallery in L...
98,98,de-en,In der Zwischenzeit kaufte die Air Force die S...,"In the meantime, the Air Force purchased the S...","In the meantime, the Air Force purchased the S..."


In [ ]:
skipped_rows

[3, 7, 10, 44, 48, 60, 66, 79, 84, 85]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_de_eng.csv"
df_results = pd.read_csv(csv_filename)
len(df_results)

100

In [ ]:

def fix_skipped_rows(target_language, df, csv_filename, skipped_rows):
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        try:
            result = generate_translation(source, target_language)
            print("translation :", result)
            # Find the start and end indices of the JSON data
            start_index = result.find('{')
            end_index = result.find('}') + 1
            result_text = result[start_index:end_index]
            result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace("', 'hyp-': '", '", "hyp-": "').replace("'}", '"}')
            print("result text ", result_text)
            #print("result text ", result_text)
            extracted_hyp = json.loads(result_text)
            print("json ", extracted_hyp)
            df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
            df.at[index, 'hyp-'] =  extracted_hyp['hyp-']
        except Exception as e:
            print(f"Skipping row {index} due to error in data extraction.")
            continue

    df.to_csv(csv_filename, index=False)


In [ ]:
fix_skipped_rows("English", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_de_eng_fixed.csv", skipped_rows )

#English-German

In [6]:
df_en_de = data["test_generation_en_de"].to_pandas()
df_en_de.head()

,id,langpair,source
0,0,en-de,He graduated from the College of Arts & Scienc...
1,1,en-de,"Most are independent nations, or associated wi..."
2,2,en-de,But there are a lot of things about birds that...
3,3,en-de,The police had trouble using their speed radar...
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t..."


In [7]:
len(df_en_de)

100

In [ ]:
df_en_de.iloc[1]["source"]

'Most are independent nations, or associated with France, and known as luxury beach resorts.'

In [ ]:
#quick test
result = generate_translation(df_de_en.iloc[1]["source"], "German")
result2 = extract_json_data(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
result2["hyp-"]

'Die meisten sind unabhängige Nationen, die sich auf die Produktion von Luxusgütern spezialisiert haben und als Strandresorts bekannt sind.'

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_de.csv"
skipped_rows = save_results("German", df_en_de, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 5 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er erwarb seinen Abschluss am College of Arts ...,Er studierte an der Universität Heidelberg und...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...",Vögel sind in Wirklichkeit Dinosaurier.
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei bemühte sich, Mr. Reid zu stoppen,..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Die Giza-Plateau, oder Giza-Nekropole im ägypt...","Die Giza-Plateau, oder Giza-Nekropole im ägypt..."
5,5,en-de,Prime Minister Stephen Harper has agreed to se...,NaN,NaN
6,6,en-de,The temple of Artemis was destroyed on July 21...,Das Artemistempel wurde am 21. Juli 356 v. Chr...,Das Artemistempel wurde am 22. Juli 356 v. Chr...
7,7,en-de,South Africa have defeated the All Blacks (New...,Südafrika hat die All Blacks (Neuseeland) in e...,"Südafrika hat die All Blacks besiegt, um die T..."
8,8,en-de,"condition is stable, though he will be isolat...","Die Bedingung ist stabil, obwohl er einige Tag...","Die Bedingung ist stabil, da er einige Tage im..."
9,9,en-de,"The world has over 5,000 different languages, ...","Die Welt hat über 5.000 verschiedene Sprachen,...","Die Welt hat etwa 5.000 Sprachen, von denen me..."


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 94 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er erwarb seinen Abschluss am College of Arts ...,Er studierte an der Universität Heidelberg und...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...",Vögel sind in Wirklichkeit Dinosaurier.
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei bemühte sich, Mr. Reid zu stoppen,..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Die Giza-Plateau, oder Giza-Nekropole im ägypt...","Die Giza-Plateau, oder Giza-Nekropole im ägypt..."
...,...,...,...,...,...
95,95,en-de,"If you invent something, Tesla invented electr...","Wenn du etwas erfunden hast, hat Tesla die ele...","Wenn du etwas erfunden hast, hat Tesla die Ele..."
96,96,en-de,"LP: The amazing thing about this is this is, I...","Das erstaunliche an dieser Sache ist, dass das...","Das erstaunliche an dieser Sache ist, dass das..."
97,97,en-de,"LP: The amazing thing about this is this is, I...","Das Faszinierende an diesem ist, dass das Syst...","Das Faszinierende an diesem ist, dass es die S..."
98,98,en-de,"""She said, """"I used to have a job but I had to...","Sie sagte, ‚Ich hatte einen Job, aber ich muss...","Sie sagte, ‚Ich verlor meinen Job, weil ich zu..."


In [ ]:
skipped_rows

[5, 10, 17, 21, 33, 45, 57, 62, 75, 85, 89, 93, 94]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_de.csv"

df_results = pd.read_csv(csv_filename)
df_results.head()

,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er erwarb seinen Abschluss am College of Arts ...,Er studierte an der Universität Heidelberg und...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...",Vögel sind in Wirklichkeit Dinosaurier.
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei bemühte sich, Mr. Reid zu stoppen,..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Die Giza-Plateau, oder Giza-Nekropole im ägypt...","Die Giza-Plateau, oder Giza-Nekropole im ägypt..."


In [ ]:
fix_skipped_rows("German", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_de_fixed.csv", skipped_rows )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : Here is the result:

{'hyp+': 'Der kanadische Premierminister Stephen Harper hat beschlossen, das "Clean Act" des Regierung vor einem parteiübergreifenden Ausschuss für eine Überprüfung zu senden, bevor es am Dienstag nach der 25-minütigen Sitzung mit dem NDP-Führer Jack Layton im PMO erneut gelesen wird.', 
'hyp-': 'Der kanadische Premierminister Stephen Harper hat beschlossen, das "Clean Act" des Regierung zu verbrennen, bevor es am Dienstag nach der 25-minütigen Sitzung mit dem NDP-Führer Jack Layton im PMO erneut gelesen wird.'}

The first hypothesis ('hyp+'), which is supported by the source text, translates the text accurately, mentioning that the Prime Minister has agreed to send the 'Clean Act' to an all-party committee for review before its second reading.

The second hypothesis ('hyp-'), which is not supported by the source text, translates the text inaccurately, stating that the Prime Minister has decided to burn the 'Clean Act' before its second reading, which

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Am 24. September 1759 unterzeichnete Arthur Guinness einen 9.000 Jahre langen Mietvertrag für die Brauerei St James' Gate in Dublin, Irland.', 'hyp-': 'Arthur Guinness kaufte die Brauerei St James' Gate in Dublin, Irland, am 24. September 1759.'}

hyp+ is supported by the source text because it accurately translates the lease duration and the brewery name.

hyp- is not supported by the source text because it incorrectly implies that Arthur Guinness bought the brewery instead of signing a lease.
result text  {"hyp+": "Am 24. September 1759 unterzeichnete Arthur Guinness einen 9.000 Jahre langen Mietvertrag für die Brauerei St James' Gate in Dublin, Irland.", "hyp-": "Arthur Guinness kaufte die Brauerei St James' Gate in Dublin, Irland, am 24. September 1759."}
json  {'hyp+': "Am 24. September 1759 unterzeichnete Arthur Guinness einen 9.000 Jahre langen Mietvertrag für die Brauerei St James' Gate in Dublin, Irland.", 'hyp-': "Arthur Guinness kaufte die Brauerei St

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

translation : {'hyp+': 'Die NSA hat ihr eigenes internes Datenformat, das sowohl das Ausgangs- als auch das Zielpunkt einer Kommunikation verfolgt, und wenn sie sagt, diese Kommunikation stammt aus Amerika, können sie dem Kongress mitteilen, wie viele dieser Kommunikationen sie heute, in diesem Moment, haben.', 'hyp-': 'Die NSA hat ein eigenes internationales Datenformat, das sowohl das Ausgangs- als auch das Zielpunkt einer Kommunikation verfolgt, und wenn sie sagt, diese Kommunikation stammt aus Europa, können sie dem Kongress mitteilen, wie viele dieser Kommunikationen sie heute, in diesem Moment, haben.'}

hyp+: This translation is supported by the source text because it accurately translates the phrase "tracks both ends of a communication" to "verfolgt, sowohl das Ausgangs- als auch das Zielpunkt einer Kommunikation", and it keeps the same meaning of the sentence.

hyp-: This translation is not supported by the source text because it changes the location from "America" to "Europa"

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': "Und schließlich, nach vier vollem Stunden meines Liegens und Starrens darauf, klingelte das Telefon, und ich schaffte es irgendwie, es abzunehmen, und es war mein Vater, und ich sagte:,,Ich stehe in ernsthafter Schwierigkeit. Wir müssen etwas unternehmen."', 'hyp-': "Und schließlich, nach vier vollem Stunden meines Lesens und Starrens darauf, klingelte das Telefon, und ich schaffte es irgendwie, es abzunehmen, und es war mein Vater, und ich sagte:,,Ich bin in ernster Not. Wir müssen uns beeilen.""}

The 'hyp+' is supported by the source text because it accurately translates the original sentence, including the idiomatic expression "lying and staring at it" which is translated to "meines Liegens und Starrens darauf", and the phrase "serious trouble" which is translated to "ernsthafter Schwierigkeit".

The 'hyp-' is not supported by the source text because it translates the phrase "lying and staring at it" to "meines Lesens und Starrens darauf", which is
result te

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Und dann gab es dieses riesige Fragezeichen. Ich liebe den Design, soweit es gilt, von diesem Ding. Und ich begann zu denken, warum habe ich es nicht geöffnet?', 'hyp-': 'Und dann war da dieses riesige Fragezeichen. Ich hasse den Design, wenn ich ehrlich bin, von diesem Ding. Und ich begann zu denken, warum ich es überhaupt erst geöffnet habe?'}

hyp+ is a translation that is supported by the source text. It maintains the same meaning and tone as the original text, and uses accurate German vocabulary and grammar.

hyp- is a translation that is not supported by the source text. It has a different tone and meaning than the original text, and uses German vocabulary and grammar that is not consistent with the source text.
result text  {"hyp+": "Und dann gab es dieses riesige Fragezeichen. Ich liebe den Design, soweit es gilt, von diesem Ding. Und ich begann zu denken, warum habe ich es nicht geöffnet?", "hyp-": "Und dann war da dieses riesige Fragezeichen. Ich hasse

In [ ]:
df_results=pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_de_fixed.csv")

In [ ]:
fix_skipped_rows("German", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_de_fixed.csv", [33, 57, 94] )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Es wurde von Willard Mack inszeniert und basiert auf einem 1917 uraufgeführten Stück, "Tiger Rose", von George Fitzmaurice.', 'hyp-': 'Es wurde von George Fitzmaurice inszeniert und basiert auf einem 1917 uraufgeführten Stück, "Tiger Rose", von Willard Mack.'}

hyp+: This translation is supported by the source text, as it accurately translates the information about the director and the basis of the play.

hyp-: This translation is not supported by the source text, as it incorrectly states that George Fitzmaurice directed the play and Willard Mack wrote it, whereas the source text actually says the opposite.
result text  {"hyp+": "Es wurde von Willard Mack inszeniert und basiert auf einem 1917 uraufgeführten Stück, 'Tiger Rose', von George Fitzmaurice.", "hyp-": "Es wurde von George Fitzmaurice inszeniert und basiert auf einem 1917 uraufgeführten Stück, 'Tiger Rose', von Willard Mack."}
json  {'hyp+': "Es wurde von Willard Mack inszeniert und basiert auf einem 19

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Für Australier ist die Vorstellung von "flauem Weiß" Kaffee unbekannt. Ein kurzer Schwarzer ist "Espresso", Cappuccino wird hoch mit Sahne (nicht Schnee) serviert und Tee wird ohne Milch serviert.', 'hyp-': 'Für Australier ist die Vorstellung von "Weißer Kaffee" unbekannt. Ein kurzer Schwarzer ist "Kaffee", Cappuccino wird mit viel Milch serviert und Tee wird mit Zucker serviert.'}

The first translation hypothesis ('hyp+') is supported by the source text, as it accurately translates the information about the differences in coffee culture between Australia and other countries.

The second translation hypothesis ('hyp-') is not supported by the source text, as it contradicts the information about the differences in coffee culture. In the source text, it is mentioned that tea is served without milk, whereas in the second translation, it is mentioned that tea is served with sugar, which is not mentioned in the source text. Additionally, the second translation also 

# English-French

In [8]:
df_en_fr = data["test_generation_en_fr"].to_pandas()
df_en_fr.head()

,id,langpair,source
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...
2,2,en-fr,"Only the effects of catastrophizing, not depre..."
3,3,en-fr,It can be customized to make it easy to read a...
4,4,en-fr,Stars give off light and heat because of the e...


In [9]:
len(df_en_fr)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr.csv"
skipped_rows = save_results("French", df_en_fr, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 1 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 4 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 90 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 99 due to error in data extraction.
finished chunk


,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate, Zagreb, Bobek a acqui...","Né à la capitale française, Paris, Bobek a acq..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,NaN,NaN
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets de catastrophisation, et non ...",Les séances de PA hebdomadaires régulières ont...
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour faciliter la le...,Il peut être adapté pour simplifier la lecture...
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN
...,...,...,...,...,...
95,95,en-fr,"William Llewelyn Williams, better known as Lle...","William Llewelyn Williams, mieux connu sous le...","William Llewelyn Williams, mieux connu sous le..."
96,96,en-fr,"Flavia Alejandra Gleske Fajin, better known as...",Flavia Gleske est une actrice et mannequin vén...,Flavia Gleske est une chanteuse française née ...
97,97,en-fr,"In the 1930s and 1940s, he was elected four ti...","Dans les années 1930 et 1940, il a été élu qua...","Dans les années 1930 et 1940, il a été élu pou..."
98,98,en-fr,In 1924 he was an invited spokesperson for the...,"En 1924, il était un porte-parole invité pour ...","En 1924, il a été un représentant officiel pou..."


In [ ]:
skipped_rows

[1,
 4,
 5,
 6,
 8,
 9,
 10,
 14,
 16,
 17,
 18,
 19,
 21,
 26,
 27,
 29,
 30,
 31,
 34,
 37,
 41,
 44,
 45,
 46,
 49,
 50,
 51,
 52,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 69,
 70,
 72,
 78,
 80,
 81,
 82,
 86,
 90,
 99]

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr.csv"

df_results = pd.read_csv(csv_filename)
df_results.head()

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate, Zagreb, Bobek a acqui...","Né à la capitale française, Paris, Bobek a acq..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,NaN,NaN
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets de catastrophisation, et non ...",Les séances de PA hebdomadaires régulières ont...
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour faciliter la le...,Il peut être adapté pour simplifier la lecture...
4,4,en-fr,Stars give off light and heat because of the e...,NaN,NaN


In [ ]:
fix_skipped_rows("French", df_results, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr_fixed.csv", skipped_rows )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Utiliser des applications de navigation GPS peut être la méthode la plus facile et la plus pratique pour se diriger lorsque vous êtes hors de votre pays d'origine.', 'hyp-': 'Se faire conduire par un chauffeur privé est la meilleure façon de se déplacer lors d'un voyage en dehors de votre pays natal.'}

The 'hyp+' translation is supported by the source text as it accurately conveys the same meaning and uses similar sentence structure and vocabulary.

The 'hyp-' translation is not supported by the source text as it suggests a different method of transportation (private driver) and does not mention GPS navigation apps, which is the main idea of the original sentence.
result text  {"hyp+": "Utiliser des applications de navigation GPS peut être la méthode la plus facile et la plus pratique pour se diriger lorsque vous êtes hors de votre pays d'origine.", "hyp-": "Se faire conduire par un chauffeur privé est la meilleure façon de se déplacer lors d'un voyage en dehor

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Les étoiles émettent de la lumière et de la chaleur en raison de l'énergie produite lorsque des atomes sont fusionnés ensemble pour former des éléments plus lourds.', 'hyp-': 'Les étoiles émettent de la lumière et de la chaleur en raison de l'énergie produite lorsque des éléments plus lourds sont divisés en atomes.'}

The translation 'hyp+' is supported by the source text because it accurately conveys the meaning of the original sentence, using the correct vocabulary and sentence structure.

The translation 'hyp-' is not supported by the source text because it contradicts the original sentence, which states that atoms are merged (or fused) together to form heavier elements, whereas 'hyp-' suggests that heavier elements are divided into atoms. This is a significant change in meaning and is not supported by the original text.
result text  {"hyp+": "Les étoiles émettent de la lumière et de la chaleur en raison de l'énergie produite lorsque des atomes sont fusionnés

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

translation : {'hyp+': 'Le personnage et le casting ont été révélés le 20 novembre, lorsque Fotiou a été mis en ligne dans un vidéo derrière les scenes vue par "Neighbours" sur leur chaîne YouTube.', 
'hyp-': 'Le personnage et le casting ont été créés le 20 novembre, lorsque Fotiou a été publié dans un livre vu par "Neighbours" sur leur bibliothèque.'}

hyp+ is supported by the source text because it accurately translates the given information about the reveal of the character and casting. The original text mentions the release of a behind-the-scenes video, which is accurately translated to "mis en ligne dans un vidéo derrière les scenes" in French. The target language is also correctly used.

hyp- is not supported by the source text because it does not accurately translate the given information. The original text does not mention the creation of the character and casting, nor does it mention a book being published. The sentence structure and wording are also incorrect.
result text  {"

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'The final line of the third verse was modified during the reign of Alexander I of Yugoslavia in "Kralja Aleksandra, Bože hrani".', 'hyp-': 'The final line of the third verse was rewritten during the reign of Alexander the Great in "Kralja Aleksandra, Bože hrani".'}

The first hypothesis ('hyp+') is supported by the source text because it accurately translates the original text and maintains the same meaning. The phrase "changed during" is correctly translated to "modified during", and the proper name "Alexander I of Yugoslavia" is left unchanged.

The second hypothesis ('hyp-') is not supported by the source text because it contains an error. Alexander the Great was a king of ancient Greece, not Yugoslavia, and the translation is incorrect. Additionally, the phrase "rewritten during" is not present in the original text, which only mentions "changed during".
result text  {"hyp+": "The final line of the third verse was modified during the reign of Alexander I of Y

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Le 30 avril 1983, un C-311 affecté à la base aéronavale de Jacksonville, en Cuba, à la baie de Guantánamo, s'est écrasé, causant la perte de 13 vies.', 'hyp-': 'Le 30 avril 1983, un C-311 a été détruit à la base aéronavale de Jacksonville, en France, causant la mort de 13 personnes.'}

The 'hyp+' is a correct translation of the source text, accurately conveying the information about the crash of the C-311 aircraft.

The 'hyp-' is an incorrect translation because it states that the crash occurred at a base in France, which is not supported by the source text. The source text specifically mentions that the aircraft was assigned to NAS Jacksonville, Cuba, and crashed at NAS Guantanamo Bay, indicating that the crash occurred in Cuba, not France.
result text  {"hyp+": "Le 30 avril 1983, un C-311 affecté à la base aéronavale de Jacksonville, en Cuba, à la baie de Guantánamo, s'est écrasé, causant la perte de 13 vies.", "hyp-": "Le 30 avril 1983, un C-311 a été détruit

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr_fixed.csv")

In [ ]:
df

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate, Zagreb, Bobek a acqui...","Né à la capitale française, Paris, Bobek a acq..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser des applications de navigation GPS pe...,Se faire conduire par un chauffeur privé est l...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets de catastrophisation, et non ...",Les séances de PA hebdomadaires régulières ont...
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour faciliter la le...,Il peut être adapté pour simplifier la lecture...
4,4,en-fr,Stars give off light and heat because of the e...,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...
...,...,...,...,...,...
95,95,en-fr,"William Llewelyn Williams, better known as Lle...","William Llewelyn Williams, mieux connu sous le...","William Llewelyn Williams, mieux connu sous le..."
96,96,en-fr,"Flavia Alejandra Gleske Fajin, better known as...",Flavia Gleske est une actrice et mannequin vén...,Flavia Gleske est une chanteuse française née ...
97,97,en-fr,"In the 1930s and 1940s, he was elected four ti...","Dans les années 1930 et 1940, il a été élu qua...","Dans les années 1930 et 1940, il a été élu pou..."
98,98,en-fr,In 1924 he was an invited spokesperson for the...,"En 1924, il était un porte-parole invité pour ...","En 1924, il a été un représentant officiel pou..."


In [ ]:
r, _ = np.where(df.isna())


In [ ]:
u, indices = np.unique(r, return_index=True)
u

array([ 8, 10, 14, 17, 18, 19, 21, 27, 29, 30, 31, 34, 41, 45, 46, 49, 52,
       59, 60, 62, 63, 65, 69, 72, 78, 80, 81, 82])

In [ ]:

def fix_skipped_rows(target_language, df, csv_filename, skipped_rows):
    skipped_rows2 = []
    for index in skipped_rows:
        row = df.loc[index]

        source = row['source']

        try:
            result = generate_translation(source, target_language)
            # Find the start and end indices of the JSON data
            start_index = result.find('{')
            end_index = result.find('}') + 1
            result_text = result[start_index:end_index]
            result_text = result_text.replace('"', "'").replace("{'hyp+': '", '{"hyp+": "').replace(".', 'hyp-': '", '.", "hyp-": "').replace(".',\n'hyp-': '", '.",\n"hyp-": "').replace(".', \n'hyp-': '", '.", \n"hyp-": "').replace("'}", '"}')
            print("result text ", result_text)
            #print("result text ", result_text)
            extracted_hyp = json.loads(result_text)
            df.at[index, 'hyp+'] =  extracted_hyp['hyp+']
            df.at[index, 'hyp-'] =  extracted_hyp['hyp-']
        except Exception as e:
            print(f"Skipping row {index} due to error in data extraction.")
            skipped_rows2.append(index)
            continue

    df.to_csv(csv_filename, index=False)
    return skipped_rows2


In [ ]:
skipped_rows = fix_skipped_rows("French", df, "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr_fixed2.csv", u)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Les dépenses de guerre de l'impérialisme américain dans la conquête ont été financées par les Philippins eux-mêmes.', 
'hyp-': 'Les dépenses de guerre de l'impérialisme américain dans la conquête ont été financées par les États-Unis.'}

hyp+ is supported by'src' because it translates "The war expenditures of U.S. imperialism" to "Les dépenses de guerre de l'impérialisme américain", and "paid for by the Filipino people themselves" to "ont été financées par les Philippins eux-mêmes". This translation is accurate and supported by the original text.

hyp- is not supported by'src' because it translates "The war expenditures of U.S. imperialism" to "Les dépenses de guerre de l'impérialisme américain", but then incorrectly translates "paid for by the Filipino people themselves" to "ont été financées par les États-Unis", which is not supported by the original text. The original text clearly states that the Filipino people paid for the war expenditures, not the United St

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'Couchsurfing a été fondé en janvier 2004 après que le programmeur informatique Casey Fenton a trouvé un vol économique pour l'Islande mais n'avait pas de logement.', 
'hyp-': 'Couchsurfing a été créé en 2004 par Casey Fenton, un jeune homme qui a découvert l'Islande et a décidé de créer une communauté pour les voyageurs.'}
result text  {"hyp+": "Couchsurfing a été fondé en janvier 2004 après que le programmeur informatique Casey Fenton a trouvé un vol économique pour l'Islande mais n'avait pas de logement.", 
"hyp-": "Couchsurfing a été créé en 2004 par Casey Fenton, un jeune homme qui a découvert l'Islande et a décidé de créer une communauté pour les voyageurs."}
json  {'hyp+': "Couchsurfing a été fondé en janvier 2004 après que le programmeur informatique Casey Fenton a trouvé un vol économique pour l'Islande mais n'avait pas de logement.", 'hyp-': "Couchsurfing a été créé en 2004 par Casey Fenton, un jeune homme qui a découvert l'Islande et a décidé de créer 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

translation : {'hyp+': '43 personnes ont été sauvées; 40 dans les embarcations de sauvetage et trois sauvées par le "Delaware".', 'hyp-': '43 personnes ont été sauvées; 40 dans les bateaux et trois sauvées par le Delaware.'}

hyp+ is supported by the source text because it accurately translates the given text into French, preserving the same structure and wording.

hyp- is not supported by the source text because it changes the wording of "lifeboats" to "bateaux", which is not present in the original text.
result text  {"hyp+": "43 personnes ont été sauvées; 40 dans les embarcations de sauvetage et trois sauvées par le 'Delaware'.", "hyp-": "43 personnes ont été sauvées; 40 dans les bateaux et trois sauvées par le Delaware."}
json  {'hyp+': "43 personnes ont été sauvées; 40 dans les embarcations de sauvetage et trois sauvées par le 'Delaware'.", 'hyp-': '43 personnes ont été sauvées; 40 dans les bateaux et trois sauvées par le Delaware.'}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : {'hyp+': 'En 1781, le gouverneur de l'Illinois, Thomas Jefferson, a promu Clark au grade de général de brigade et lui a confié le commandement de toutes les milices dans les comtés du Kentucky et de la Virginie.', 'hyp-': 'En 1781, Thomas Jefferson a créé Clark, gouverneur de l'Illinois, et lui a confié le commandement de toutes les milices dans les comtés du Kentucky et de la Virginie.'}

hyp+ is a translation that is supported by the source text, as it accurately conveys the same information.

hyp- is a translation that is not supported by the source text, as it incorrectly states that Thomas Jefferson created Clark as the governor of Illinois, when in fact, Clark was promoted to brigadier general by Jefferson and given command of the militia in Kentucky and Virginia counties, not created as governor.
result text  {"hyp+": "En 1781, le gouverneur de l'Illinois, Thomas Jefferson, a promu Clark au grade de général de brigade et lui a confié le commandement de toutes les m

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


translation : Here are the translation hypotheses:

{'hyp+': 'Il est proche du lac Corrib, sur la route N59 menant à Oughterard et Clifden, en Connemara.', 
'hyp-': 'Il est à côté du lac Corrib, sur la route N59 vers Paris et Lyon, en Connemara.'}

The 'hyp+' translation is supported by the source text because it accurately conveys the location and direction of the mentioned road. The 'hyp-' translation is not supported because it provides incorrect information about the direction of the road and the cities mentioned.
result text  {"hyp+": "Il est proche du lac Corrib, sur la route N59 menant à Oughterard et Clifden, en Connemara.", 
"hyp-": "Il est à côté du lac Corrib, sur la route N59 vers Paris et Lyon, en Connemara."}
json  {'hyp+': 'Il est proche du lac Corrib, sur la route N59 menant à Oughterard et Clifden, en Connemara.', 'hyp-': 'Il est à côté du lac Corrib, sur la route N59 vers Paris et Lyon, en Connemara.'}
translation : {'hyp+': 'Le caractère et le casting ont été révélés

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_en_fr_fixed2.csv")
len(df)

100

# French-English

In [12]:
df_fr_en = data["test_generation_fr_en"].to_pandas()
df_fr_en.head()

,id,langpair,source
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...
1,1,fr-en,"est relativement facile en italien, car la pl..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...
3,3,fr-en,Northern avait sollicité une aide en raison de...
4,4,fr-en,"Au fil du temps, à mesure que la population s'..."


In [11]:
len(df_fr_en)

100

In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_fr_en.csv"
skipped_rows = save_results("English", df_fr_en, csv_filename)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council c...,Martelly has been elected to a new Electoral C...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is extremely difficult in Italian, because ..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism is a form of magic that allows peopl...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern sought help due to its exposure to th...,Northern caused the 2007 subprime crisis and t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","As time goes by, the population adapts to its ...",The population is transformed into a completel...
5,5,fr-en,L'archipel est un État en libre association av...,The archipelago is a free-associated state wit...,The archipelago is a country in the North Atla...
6,6,fr-en,"Bien que Goma soit raisonnablement sûre, toute...","Although Goma is relatively safe, all visits m...",Goma is not safe and it is recommended to avoi...
7,7,fr-en,Donnez également des exemplaires de la régleme...,Also provide examples of the regulation and co...,Give also some examples of the regulation and ...
8,8,fr-en,Parmi les sports populaires figurent le footba...,"Among popular sports are football, basketball,...","Among popular sports are tennis, swimming, and..."
9,9,fr-en,"La société de gestion du groupe, HK Management...",HK Management Inc did not provide an initial r...,HK Management Inc provided a detailed explanat...


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 11 due to error in data extraction.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Skipping row 95 due to error in data extraction.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


finished chunk


,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council c...,Martelly has been elected to a new Electoral C...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is extremely difficult in Italian, because ..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism is a form of magic that allows peopl...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern sought help due to its exposure to th...,Northern caused the 2007 subprime crisis and t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","As time goes by, the population adapts to its ...",The population is transformed into a completel...
...,...,...,...,...,...
95,95,fr-en,"La couleur de capuchon varie, de marron à jaun...",NaN,NaN
96,96,fr-en,La rivière Arieşul Mare est un affluent de la ...,The Arieşul Mare River is a tributary of the V...,The Arieşul Mare River is a source of the Vâlc...
97,97,fr-en,"Born est né en Allemagne de parents juifs, le ...","Born was born in Germany to Jewish parents, th...","Born was born in France to Catholic parents, t..."
98,98,fr-en,"À proximité se trouve la rivière Wallowa, un a...","Nearby lies the Wallowa River, a tributary of ...",The Wallowa River is located in the center of ...


In [ ]:
skipped_rows

[11, 35, 36, 41, 42, 48, 60, 74, 75, 82, 83, 86, 89, 90, 95]

In [ ]:
# fix skipped rows
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_fr_en.csv")

In [ ]:
df.head()

,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council c...,Martelly has been elected to a new Electoral C...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is extremely difficult in Italian, because ..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism is a form of magic that allows peopl...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern sought help due to its exposure to th...,Northern caused the 2007 subprime crisis and t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","As time goes by, the population adapts to its ...",The population is transformed into a completel...


In [ ]:
csv_filename = "/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/results_llama3_fr_en_fixed.csv"
skipped_rows = fix_skipped_rows("English", df, csv_filename, skipped_rows)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "The problem started on January 1st when dozens of local residents began complaining to the post office in Obanazawa that they had not received their traditional and regular New Year's cards.", "hyp-": "The problem started on January 1st when dozens of aliens began complaining to the post office in Obanazawa that they had not received their traditional and regular New Year's cards."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "In 2012, Gil joined the television program 'Remake of the Salvador - Royales - Films 'Mundo Man ay Magunaw' playing the role of Jennifer la Pena.", 
"hyp-": "In 2012, Gil was a part of the 'Remake of the Salvador - Royales - Films 'Mundo Man ay Magunaw' production team."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

result text  {"hyp+": "The admiral rejected the request until it was too late, and the 'Cavour' had to use a deeper sandbank on November 12th at 04:30 am.", "hyp-": "The admiral accepted the request, and the 'Cavour' was able to navigate the shallow waters on November 12th at 04:30 am."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "Daniil Ostrogski also declares that Dmytro is the father of Bilinsky, also known as 'Danylo Dmytrovych'.", 
"hyp-": "Daniil Ostrogski claims that Bilinsky is the son of Dmytro."}


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


result text  {"hyp+": "This requires continuous monitoring of real partial pressures over time and, for maximum efficiency, real-time processing by the diver's computer decompression unit.", 
"hyp-": "This requires continuous monitoring of real partial pressures over time and, for maximum efficiency, offline processing by the diver's computer decompression unit."}
result text  {"hyp+": "The capuchin's hat color varies, from brown to yellow, often with a brown spot on the hat at maturity.", 
"hyp-": "The capuchin's hat is usually red and has a blue stripe."}


In [ ]:
skipped_rows

[]

## Find skipped rows

In [ ]:

def find_skipped_rows(df):
  r, _ = np.where(df.isna())
  u, indices = np.unique(r, return_index=True)
  return u



In [ ]:
## English-german
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_de_fixed.csv")
u = find_skipped_rows(df)
print(u)


[]


In [ ]:
## German-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_de_eng_fixed.csv")
u = find_skipped_rows(df)
print(u)


[]


In [ ]:
## French-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_fr_en_fixed.csv")
u = find_skipped_rows(df)
print(u)


[]


In [ ]:
## English-French
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_fr_fixed.csv")
u = find_skipped_rows(df)
print(u)


[52]


In [ ]:
translation = generate_translation(df.iloc[52]["source"], "French")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(translation)

{'hyp+': 'L\'américain pionnier John Sutter (1803-1880) arriva en Alta-Californie avec d\'autres colons euro-suisses en août 1839.', 
'hyp-': 'John Sutter, un pionnier américain, est arrivé en Californie en 1839 avec d\'autres Européens.'}

The first hypothesis, 'hyp+', is supported by the source text because it accurately translates the sentence, including the use of the correct title for the region "Alta California" and the inclusion of the other settlers as "Euro-Swiss".

The second hypothesis, 'hyp-', is not supported by the source text because it omits the specific title "Alta California" and incorrectly generalizes the nationality of the settlers as "Européens" instead of specifying "Euro-Swiss".


## Remove extra columns

In [ ]:
# English-German
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_de_fixed.csv")
df.head()

,id,langpair,source,hyp+,hyp-
0,0,en-de,He graduated from the College of Arts & Scienc...,Er erwarb seinen Abschluss am College of Arts ...,Er studierte an der Universität Heidelberg und...
1,1,en-de,"Most are independent nations, or associated wi...",Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,en-de,But there are a lot of things about birds that...,"Aber es gibt viele Dinge über Vögel, die wie e...",Vögel sind in Wirklichkeit Dinosaurier.
3,3,en-de,The police had trouble using their speed radar...,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei bemühte sich, Mr. Reid zu stoppen,..."
4,4,en-de,"""The Giza Plateau, or """"Giza Necropolis"""" in t...","Die Giza-Plateau, oder Giza-Nekropole im ägypt...","Die Giza-Plateau, oder Giza-Nekropole im ägypt..."


In [ ]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,Er erwarb seinen Abschluss am College of Arts ...,Er studierte an der Universität Heidelberg und...
1,1,Die meisten sind unabhängige Nationen oder ass...,Die meisten sind unabhängige Nationen oder ass...
2,2,"Aber es gibt viele Dinge über Vögel, die wie e...",Vögel sind in Wirklichkeit Dinosaurier.
3,3,"Die Polizei hatte Schwierigkeiten, ihre Geschw...","Die Polizei bemühte sich, Mr. Reid zu stoppen,..."
4,4,"Die Giza-Plateau, oder Giza-Nekropole im ägypt...","Die Giza-Plateau, oder Giza-Nekropole im ägypt..."


In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_de_final.csv", mode='w', header=True, index=False)


In [ ]:
## German-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_de_eng_fixed.csv")
df.head()


,id,langpair,source,hyp+,hyp-
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...",Luxury guesthouses are now a common occurrence...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in Britain, but the Sc...","This is common practice in Britain, but the Sc..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"Based on this fossil, it can be said that the ...",This fossil is the key to understanding the ti...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...",A minesweeper of the Avenger-class was en rout...,The minesweeper was on a mission to destroy en...


In [ ]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,"In countries, offering luxury guesthouses has ...",Luxury guesthouses are now a common occurrence...
1,1,"This is common practice in Britain, but the Sc...","This is common practice in Britain, but the Sc..."
2,2,"Based on this fossil, it can be said that the ...",This fossil is the key to understanding the ti...
3,3,"We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,A minesweeper of the Avenger-class was en rout...,The minesweeper was on a mission to destroy en...


In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_de_en_final.csv", mode='w', header=True, index=False)

In [ ]:
## French-English
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_fr_en_fixed.csv")
df.head()

,id,langpair,source,hyp+,hyp-
0,0,fr-en,Martelly a intronisé hier un nouveau Conseil é...,Martelly inaugurated a new Electoral Council c...,Martelly has been elected to a new Electoral C...
1,1,fr-en,"est relativement facile en italien, car la pl...","It is relatively easy in Italian, because most...","It is extremely difficult in Italian, because ..."
2,2,fr-en,Le mysticisme peut être distingué des autres f...,Mysticism can be distinguished from other form...,Mysticism is a form of magic that allows peopl...
3,3,fr-en,Northern avait sollicité une aide en raison de...,Northern sought help due to its exposure to th...,Northern caused the 2007 subprime crisis and t...
4,4,fr-en,"Au fil du temps, à mesure que la population s'...","As time goes by, the population adapts to its ...",The population is transformed into a completel...


In [ ]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,Martelly inaugurated a new Electoral Council c...,Martelly has been elected to a new Electoral C...
1,1,"It is relatively easy in Italian, because most...","It is extremely difficult in Italian, because ..."
2,2,Mysticism can be distinguished from other form...,Mysticism is a form of magic that allows peopl...
3,3,Northern sought help due to its exposure to th...,Northern caused the 2007 subprime crisis and t...
4,4,"As time goes by, the population adapts to its ...",The population is transformed into a completel...


In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_fr_en_final.csv", mode='w', header=True, index=False)


In [ ]:
## English-French
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_fr_fixed.csv", sep=';')

In [ ]:
df.head()

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate, Zagreb, Bobek a acqui...","Né à la capitale française, Paris, Bobek a acq..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser des applications de navigation GPS pe...,Se faire conduire par un chauffeur privé est l...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets de catastrophisation, et non ...",Les séances de PA hebdomadaires régulières ont...
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour faciliter la le...,Il peut être adapté pour simplifier la lecture...
4,4,en-fr,Stars give off light and heat because of the e...,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...


In [ ]:
df.head()

,id,langpair,source,hyp+,hyp-
0,0,en-fr,"Born in the Croatian capital, , Bobek gained f...","Né à la capitale croate,, Bobek est devenu cél...","Né à la capitale croate, Zagreb, Bobek est dev..."
1,1,en-fr,Using GPS navigation apps can be the easiest a...,Utiliser des applications de navigation GPS pe...,Utiliser des applications de navigation GPS pe...
2,2,en-fr,"Only the effects of catastrophizing, not depre...","Seuls les effets du catastrophisme, et non la ...","Seuls les effets du catastrophisme, et non la ..."
3,3,en-fr,It can be customized to make it easy to read a...,Il peut être personnalisé pour qu'il soit faci...,Il peut être personnalisé pour qu'il soit faci...
4,4,en-fr,Stars give off light and heat because of the e...,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...


In [ ]:
df = df.drop(['langpair', 'source'], axis=1)
df.head()

,id,hyp+,hyp-
0,0,"Né à la capitale croate, Zagreb, Bobek a acqui...","Né à la capitale française, Paris, Bobek a acq..."
1,1,Utiliser des applications de navigation GPS pe...,Se faire conduire par un chauffeur privé est l...
2,2,"Seuls les effets de catastrophisation, et non ...",Les séances de PA hebdomadaires régulières ont...
3,3,Il peut être personnalisé pour faciliter la le...,Il peut être adapté pour simplifier la lecture...
4,4,Les étoiles émettent de la lumière et de la ch...,Les étoiles émettent de la lumière et de la ch...


In [ ]:
df.to_csv("/content/drive/MyDrive/Master Project-Hallucination/Group1_results/Generation_task/Llama3/prompt1/results_llama3_en_fr_final.csv", mode='w', header=True, index=False)